In [ ]:
---
layout: post
title: Full Stack(Project) Feature Blog(PPR)
type: issues
comments: true
---

## CPT Requirements for GET Method

```python
def get(self):
        category = request.args.get('category', 'general')
        hobbies = Hobby.query.filter_by(category=category).all()
        if hobbies:
            return jsonify({"category": category, "hobbies": [hobby.name for hobby in hobbies]})
        else:
            return jsonify({"message": "Category not found"}), 404
```


| Requirement                                         | Met? | Explanation |
|-----------------------------------------------------|------|-------------|
| **User Input**                                      | ✅   | The method takes user input via `request.args.get('category', 'general')`, retrieving a category from the query parameters. |
| **A List**                                         | ✅   | The query `hobbies = Hobby.query.filter_by(category=category).all()` returns a list of hobby objects. Additionally, `[hobby.name for hobby in hobbies]` is a list comprehension. |
| **A Procedure**                                    | ✅   | The `get(self)` method is a defined procedure that executes a specific task (retrieving and returning hobby data). |
| **A Call to the Procedure**                        | ✅   | The method is called when an HTTP GET request is made to fetch hobbies. |
| **An Algorithm with Sequencing, Selection, and Iteration** | ✅   | **Sequencing**: The method follows a logical order: retrieving input, querying data, checking conditions, and returning a response. <br> **Selection**: The `if hobbies:` condition checks if hobbies exist and determines whether to return results or an error message. <br> **Iteration**: The list comprehension `[hobby.name for hobby in hobbies]` loops through the `hobbies` list. |
| **Instruction for Output Based on Input and Program Functionality** | ✅   | The method returns a JSON response that changes based on the input category—either a list of hobbies or an error message if no category is found. |

### **Final Verdict:** ✅ The GET method fully satisfies all CPT requirements.

## Full Stack Primary Features

### Backend API(Flask)

```python
class HobbyResource(Resource):
    def get(self):
        category = request.args.get('category', 'general')
        hobbies = Hobby.query.filter_by(category=category).all()
        if hobbies:
            return jsonify({"category": category, "hobbies": [hobby.name for hobby in hobbies]})
        else:
            return jsonify({"message": "Category not found"}), 404
    
    def post(self):
        data = request.get_json()
        if not data or not data.get('name') or not data.get('category'):
            return jsonify({"message": "Hobby name and category are required"}), 400
        
        hobby = Hobby(name=data['name'], category=data['category'])
        if hobby.create():
            return jsonify({"message": "Hobby created", "hobby": hobby.name, "category": hobby.category})
        else:
            return jsonify({"message": "Error creating hobby"}), 500

    def put(self):
        data = request.get_json()
        if not data or not data.get('name') or not data.get('category') or not data.get('old_name'):
            return jsonify({"message": "Hobby name, old name, and category are required to update"}), 400
        
        hobby = Hobby.query.filter_by(name=data['old_name'], category=data['category']).first()
        if not hobby:
            return jsonify({"message": "Hobby not found"}), 404
        
        hobby.name = data['name']
        if hobby.update():
            return jsonify({"message": "Hobby updated", "old_name": data['old_name'], "new_name": hobby.name})
        else:
            return jsonify({"message": "Error updating hobby"}), 500

    def delete(self):
        data = request.get_json()
        if not data or not data.get('name') or not data.get('category'):
            return jsonify({"message": "Hobby name and category are required to delete"}), 400
        
        hobby = Hobby.query.filter_by(name=data['name'], category=data['category']).first()
        if not hobby:
            return jsonify({"message": "Hobby not found"}), 404
        
        if hobby.delete():
            return jsonify({"message": "Hobby deleted", "name": hobby.name})
        else:
            return jsonify({"message": "Error deleting hobby"}), 500
```
- Using Flask, a RESTful API
- GET, POST, PUT, DELETE methods all function in the API for full CRUD functionality

### Backend Database(SQLAlchemy)

```python
class Hobby(db.Model):
    __tablename__ = 'hobby'
    
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    category = db.Column(db.String(100), nullable=False)

    def __init__(self, name, category):
        """
        Initialize a Hobby object.
        
        Args:
            name (str): The name of the hobby.
            category (str): The category of the hobby.
        """
        self.name = name
        self.category = category

    def create(self):
        """
        Create a new hobby in the database.
        
        Returns:
            bool: True if the hobby was successfully created, False otherwise.
        """
        try:
            db.session.add(self)
            db.session.commit()
            return True
        except IntegrityError as e:
            logging.error(f"Error creating hobby: {e}")
            db.session.rollback()
            return False

    def update(self):
        """
        Update an existing hobby in the database.
        
        Returns:
            bool: True if the hobby was successfully updated, False otherwise.
        """
        try:
            db.session.commit()
            return True
        except IntegrityError as e:
            logging.error(f"Error updating hobby: {e}")
            db.session.rollback()
            return False

    def delete(self):
        """
        Delete an existing hobby from the database.
        
        Returns:
            bool: True if the hobby was successfully deleted, False otherwise.
        """
        try:
            db.session.delete(self)
            db.session.commit()
            return True
        except IntegrityError as e:
            logging.error(f"Error deleting hobby: {e}")
            db.session.rollback()
            return False

    def read(self):
        """
        Read the hobby data.
        
        Returns:
            dict: A dictionary representation of the hobby.
        """
        return {
            "id": self.id,
            "name": self.name,
            "category": self.category
        }

    @staticmethod
    def restore(data):
        """
        Restore hobbies from a list of dictionaries, replacing existing entries.

        Args:
            data (list): List of dictionaries containing hobby data.

        Returns:
            dict: Dictionary of restored Hobby objects.
        """
        with app.app_context():
            # Clear the existing table
            db.session.query(Hobby).delete()
            db.session.commit()

            restored_hobbies = {}
            for hobby_data in data:
                hobby = Hobby(
                    name=hobby_data['name'],
                    category=hobby_data['category']
                )
                hobby.create()
                restored_hobbies[hobby_data['id']] = hobby

            return restored_hobbies

def initHobbies():
    """
    The initHobbies function creates the Hobby table and adds tester data to the table.
    
    Uses:
        The db ORM methods to create the table.
    
    Instantiates:
        Hobby objects with tester data.
    
    Raises:
        IntegrityError: An error occurred when adding the tester data to the table.
    """
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        
        """Tester data for Hobby table"""
        hobbies = [
            {"name": "Reading", "category": "general"},
            {"name": "Writing", "category": "general"},
            {"name": "Football", "category": "sports"},
            {"name": "Basketball", "category": "sports"},
            {"name": "Painting", "category": "arts"},
            {"name": "Drawing", "category": "arts"},
        ]

        for hobby_data in hobbies:
            hobby = Hobby(name=hobby_data["name"], category=hobby_data["category"])
            try:
                db.session.add(hobby)
                db.session.commit()
            except IntegrityError as e:
                logging.error(f"Error creating hobby: {e}")
                db.session.rollback()

        print("Database has been initialized and populated with initial hobby data.")
```
- Implements a database table using SQLAlchemy, initialized through the ```init()``` function with columns ```id```, ```name```, and ```category```.
- Implements static data into the database through the ```initHobbies()``` function.
- Implements restore functionality for the database through a json file and through the ```restore()``` function.

### Frontend Fetch(JavaScript)
```js
import { pythonURI, fetchOptions } from '{{ site.baseurl }}/assets/js/api/config.js';

    async function fetchHobbies() {
        try {
            const category = document.getElementById('category').value;
            const response = await fetch(`${pythonURI}/api/hobby?category=${category}`, {
                ...fetchOptions,
                method: 'GET'
            });

            if (!response.ok) {
                throw new Error('Failed to fetch hobbies: ' + response.statusText);
            }
            const data = await response.json();
            const hobbiesList = document.getElementById('hobbies-list');
            hobbiesList.innerHTML = "";

            data.hobbies.forEach(hobby => {
                const listItem = document.createElement('li');
                listItem.style.display = 'flex';
                listItem.style.alignItems = 'center';
                listItem.style.justifyContent = 'space-between';

                const hobbyText = document.createElement('span');
                hobbyText.textContent = hobby;

                const updateButton = document.createElement('button');
                updateButton.textContent = 'Update';
                updateButton.style.marginLeft = '10px'; // Add some space between buttons
                updateButton.style.padding = '2px 5px';
                updateButton.style.fontSize = '12px'; // Make the button smaller
                updateButton.style.width = '60px'; // Make the button smaller horizontally
                updateButton.classList.add('update-btn');
                updateButton.onclick = () => promptUpdateHobby(hobby, category);

                const deleteButton = document.createElement('button');
                deleteButton.textContent = 'Delete';
                deleteButton.style.marginLeft = '10px'; // Add some space between buttons
                deleteButton.style.padding = '2px 5px';
                deleteButton.style.fontSize = '12px'; // Make the button smaller
                deleteButton.style.width = '60px'; // Make the button smaller horizontally
                deleteButton.onclick = () => deleteHobby(hobby, category);

                listItem.appendChild(hobbyText);
                listItem.appendChild(updateButton);
                listItem.appendChild(deleteButton);
                hobbiesList.appendChild(listItem);
            });
        } catch (error) {
            console.error('Error fetching hobbies:', error);
        }
    }

    async function addHobby() {
        const hobbyName = document.getElementById('new-hobby-name').value;
        const category = document.getElementById('new-hobby-category').value;
        const allHobbiesResponse = await fetch(`${pythonURI}/api/hobby`, {
            ...fetchOptions,
            method: 'GET'
        });

        if (!allHobbiesResponse.ok) {
            alert('Failed to fetch all hobbies');
            return;
        }

        const allHobbiesData = await allHobbiesResponse.json();
        const allHobbiesList = allHobbiesData.hobbies.flatMap(hobby => hobby);

        if (allHobbiesList.includes(hobbyName)) {
            alert('Hobby already exists!');
            return;
        }

        try {
            const response = await fetch(`${pythonURI}/api/hobby`, {
                ...fetchOptions,
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ name: hobbyName, category: category })
            });

            if (!response.ok) {
                throw new Error('Failed to add hobby: ' + response.statusText);
            }

            alert('Hobby added successfully!');
            document.getElementById('new-hobby-name').value = ''; // Clear input
            fetchHobbies(); // Refresh hobbies list
        } catch (error) {
            console.error('Error adding hobby:', error);
            alert('Error adding hobby: ' + error.message);
        }
    }

    async function promptUpdateHobby(hobbyName, category) {
        const updatedHobbyName = prompt('Enter new hobby name:', hobbyName);
        const allHobbiesResponse = await fetch(`${pythonURI}/api/hobby`, {
            ...fetchOptions,
            method: 'GET'
        });

        if (!allHobbiesResponse.ok) {
            alert('Failed to fetch all hobbies');
            return;
        }

        const allHobbiesData = await allHobbiesResponse.json();
        const allHobbiesList = allHobbiesData.hobbies.flatMap(hobby => hobby);

        if (allHobbiesList.includes(updatedHobbyName)) {
            alert('Hobby already exists!');
            return;
        }

        if (updatedHobbyName) {
            try {
                const response = await fetch(`${pythonURI}/api/hobby`, {
                    ...fetchOptions,
                    method: 'PUT',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({ old_name: hobbyName, name: updatedHobbyName, category: category })
                });

                if (!response.ok) {
                    throw new Error('Failed to update hobby: ' + response.statusText);
                }

                alert('Hobby updated successfully!');
                fetchHobbies(); // Refresh hobbies list
            } catch (error) {
                console.error('Error updating hobby:', error);
                alert('Error updating hobby: ' + error.message);
            }
        }
    }

    async function deleteHobby(hobbyName, category) {
        const confirmation = confirm('Are you sure you want to delete this hobby?');
        if (!confirmation) {
            return;
        }

        try {
            const response = await fetch(`${pythonURI}/api/hobby`, {
                ...fetchOptions,
                method: 'DELETE',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ name: hobbyName, category: category })
            });

            if (!response.ok) {
                throw new Error('Failed to delete hobby: ' + response.statusText);
            }

            alert('Hobby deleted successfully!');
            fetchHobbies(); // Refresh hobbies list
        } catch (error) {
            console.error('Error deleting hobby:', error);
            alert('Error deleting hobby: ' + error.message);
        }
    }

    document.getElementById('category').addEventListener('change', fetchHobbies);
    document.getElementById('add-hobby-btn').addEventListener('click', addHobby);
    fetchHobbies();
```
- Implements CRUD functionality into the frontend by fetching the API from the backend
- Implements some styling of the update and delete buttons in the UI
- Parses JSON from API into JavaScript object and DOM so it can be displayed in the site

### Next Steps
- Implement a program that allows only admins to delete hobbies from the hobby log
- Make the hobbies log more dynamic by allowing users to click on a hobby and favorite it for future reference